### 0. Environment setup (kelvin2)

In [ ]:
# 00_environment
# i. create conda environment in NI-HPC kelvin2

conda env create -f regenie-env.yml

conda activate regenie-env

### 1. Data prep & qc

In [ ]:
# 01_data_prep_qc
# x. Liftover

# try without liftover


In [ ]:
%%bash

# 01_data_prep_qc
# i. PLINK QC for Rare Variants

##############################################################
# USER-DEFINED INPUTS (EDIT HERE) #
##############################################################

BFILE="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/4. Gene Burden/01_data_prep_qc_ADNI1/inputs/ADNI_cluster_01_forward_757LONI"
OUTPUT="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/4. Gene Burden/01_data_prep_qc_ADNI1/outputs/ADNI1_preQC"


##############################################################
# DO NOT EDIT BELOW THIS LINE #
##############################################################


# ── QC rare variants ─────────────────────────────

plink --bfile "$BFILE" \
      --maf 0.005 \
      --max-maf 0.01 \
      --geno 0.05 \
      --mind 0.05 \
      --hwe 1e-6 \
      --mac 1 \
      --not-chr 0 \
      --output-chr MT \
      --make-bed \
      --out "$OUTPUT"
STATUS=$?

if [ $STATUS -ne 0 ]; then
  echo "❌ ERROR: PLINK QC failed with exit code $STATUS"
  exit $STATUS
fi

echo "✅ Files saved: ${OUTPUT}.[bed|bim|fam]"
echo "🏁 QC (rare) complete."

In [ ]:
%%bash

# 01_data_prep_qc
# ii. PLINK QC for Common Variants (to create null model for regenie step 1)

##############################################################
# USER-DEFINED INPUTS (EDIT HERE) #
##############################################################

BFILE="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/4. Gene Burden/01_data_prep_qc_ADNI1/inputs/ADNI_cluster_01_forward_757LONI"

OUTPUT="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/4. Gene Burden/01_data_prep_qc_ADNI1/outputs/ADNI1_preQC_common"

##############################################################
# DO NOT EDIT BELOW THIS LINE #
##############################################################

# ── QC common variants ─────────────────────────────

plink --bfile "$BFILE" \
      --maf 0.01 \
      --geno 0.05 \
      --mind 0.05 \
      --hwe 1e-6 \
      --mac 1 \
      --not-chr 0 \
      --output-chr MT \
      --make-bed \
      --out "$OUTPUT"
STATUS=$?

if [ $STATUS -ne 0 ]; then
  echo "❌ ERROR: PLINK QC failed with exit code $STATUS"
  exit $STATUS
fi

echo "✅ Files saved: ${OUTPUT}.[bed|bim|fam]"
echo "🏁 QC (common) complete."

In [ ]:
%%bash

# 01_data_prep_qc
# iii. PLINK LD pruning common variants set

##############################################################
# USER-DEFINED INPUTS (EDIT HERE) #
##############################################################

BFILE="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/4. Gene Burden/01_Data_Prep_QC_ADNI1/outputs/ADNI1_preQC_common"

OUTPUT="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/4. Gene Burden/01_Data_Prep_QC_ADNI1/outputs/ADNI1_pruned"

##############################################################
# DO NOT EDIT BELOW THIS LINE #
##############################################################

plink \
  --bfile "$BFILE" \
  --indep-pairwise 200 50 0.3 \
  --out "$OUTPUT"
STATUS=$?

if [ $STATUS -ne 0 ]; then
  echo "❌ ERROR: PLINK QC failed with exit code $STATUS"
  exit $STATUS
fi

echo "✅ Files saved: ${OUTPUT}"
echo "🏁 LD pruning complete.

In [ ]:
%%bash

# 01_data_prep_qc
# iv. PLINK PCA generation on common variants set

##############################################################
# USER-DEFINED INPUTS (EDIT HERE) #
##############################################################

BFILE="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/4. Gene Burden/01_Data_Prep_QC_ADNI1/outputs/ADNI1_preQC_common"

PRUNED="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/4. Gene Burden/01_Data_Prep_QC_ADNI1/outputs/ADNI1_pruned.prune.in"

OUTPUT="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/4. Gene Burden/01_Data_Prep_QC_ADNI1/outputs/ADNI1_PCA"

##############################################################
# DO NOT EDIT BELOW THIS LINE #
##############################################################

plink \
  --bfile "$BFILE" \
  --extract "$PRUNED" \
  --pca 10 header \
  --out "$OUTPUT"
STATUS=$?

if [ $STATUS -ne 0 ]; then
  echo "❌ ERROR: PLINK QC failed with exit code $STATUS"
  exit $STATUS
fi

echo "✅ Files saved: ${OUTPUT}"
echo "🏁 PCA complete."

In [ ]:
# 01_Data_Prep_QC
# v. Generate Phenotype and Covariate Files
# Python script

##############################################################
# USER-DEFINED INPUTS (EDIT HERE) #
##############################################################

# Paths to input files
ADNI_MERGE_PATH = r"C:\Users\B00731414\OneDrive - Ulster University\6. Code\ADNI\4. Gene Burden\01_data_prep_qc_ADNI1\inputs\ADNIMERGE_15May2025.csv"

PLINK_FAM_PATH  = r"C:\Users\B00731414\OneDrive - Ulster University\6. Code\ADNI\4. Gene Burden\01_data_prep_qc_ADNI1\outputs\ADNI1_preQC.fam"

# Path to output directory
OUTPUT_DIR = Path(r"C:\Users\B00731414\OneDrive - Ulster University\6. Code\ADNI\4. Gene Burden\01_data_prep_qc_ADNI1\outputs")

##############################################################
# DO NOT EDIT BELOW THIS LINE #
##############################################################

from pathlib import Path
import pandas as pd

def main():
    print("📥 Loading ADNI phenotype and PLINK sample data...")
    # Load ADNI phenotype data
    adni_data = pd.read_csv(ADNI_MERGE_PATH, low_memory=False)
    # Load PLINK .fam file
    plink_fam = pd.read_csv(
        PLINK_FAM_PATH, sep=r'\s+', header=None,
        names=['FID', 'IID', 'PAT', 'MAT', 'SEX', 'PHENOTYPE']
    )

# ── Filter to baseline AD vs CN ─────────────────────────────

    ad_cn_data = adni_data[
        (adni_data['VISCODE'] == 'bl') &
        (adni_data['DX_bl'].isin(['AD', 'CN']))
    ].copy()
    print(f"🔍 Baseline AD vs CN: {len(ad_cn_data)} samples (before matching)")

# ── Match PTID to IID and create FID mapping ───────────────

    ad_cn_data['IID'] = ad_cn_data['PTID'].astype(str)
    ad_cn_data = ad_cn_data[ad_cn_data['IID'].isin(plink_fam['IID'].astype(str))]
    iid_to_fid = dict(zip(plink_fam['IID'].astype(str), plink_fam['FID']))
    ad_cn_data['FID'] = ad_cn_data['IID'].map(iid_to_fid)

    if ad_cn_data['FID'].isnull().any():
        missing = ad_cn_data[ad_cn_data['FID'].isnull()]['IID'].tolist()
        print("⚠️ WARNING: Some samples could not be mapped to FIDs")
        print(f"Missing FID for: {missing}")

    print(f"✅ After genetic match: {len(ad_cn_data)} samples => "
          f"{(ad_cn_data['DX_bl']=='AD').sum()} AD, {(ad_cn_data['DX_bl']=='CN').sum()} CN")

# ── Create phenotype & covars ───────────────

    ad_cn_data['AD_vs_CN'] = ad_cn_data['DX_bl'].map({'AD':1, 'CN':0})
    ad_cn_data['sex']       = ad_cn_data['PTGENDER'].map({'Male':1, 'Female':0})
    ad_cn_data['age']       = ad_cn_data['AGE']
    ad_cn_data['education'] = ad_cn_data['PTEDUCAT']
    ad_cn_data['apoe4_count']=ad_cn_data['APOE4']

    phenotype_cols = ['FID','IID','AD_vs_CN','age','sex','education','apoe4_count','PTID','DX_bl']
    pheno_df = ad_cn_data[phenotype_cols].dropna(
        subset=['AD_vs_CN','age','sex','education','apoe4_count']
    )
    print(f"🔍 Final sample size: {len(pheno_df)} "
          f"({(pheno_df['AD_vs_CN']==1).sum()} AD, {(pheno_df['AD_vs_CN']==0).sum()} CN)")

# ── Check FID alignment with FAM file ───────────────

    print("\n🔧 Verifying FID alignment with FAM file...")
    fam_sample  = plink_fam[plink_fam['IID'].isin(pheno_df['IID'])][['FID','IID']].head()
    pheno_sample= pheno_df[['FID','IID']].head()
    print("FAM file FID/IID mapping (sample):"); print(fam_sample)
    print("\nPhenotype file FID/IID mapping (sample):"); print(pheno_sample)

    merged = pheno_df.merge(
        plink_fam[['FID','IID']], on='IID', suffixes=('_pheno','_fam')
    )
    if (merged['FID_pheno']==merged['FID_fam']).all():
        print("✅ FID alignment check: PASS")
    else:
        mismatches = merged[merged['FID_pheno']!=merged['FID_fam']]
        print("❌ FID alignment check: FAIL")
        print(f"Number of mismatches: {len(mismatches)}")

# ── Save outputs ───────────────

    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    pheno_df[['FID','IID','AD_vs_CN']].to_csv(
        OUTPUT_DIR/"ADNI_AD_vs_CN_phenotypes.txt", sep='\t',
        index=False, na_rep='NA'
    )
    pheno_df[['FID','IID','age','sex','education','apoe4_count']].to_csv(
        OUTPUT_DIR/"ADNI_AD_vs_CN_covariates.txt", sep='\t',
        index=False, na_rep='NA'
    )
    pheno_df.to_csv(
        OUTPUT_DIR/"ADNI_AD_vs_CN_summary.txt", sep='\t',
        index=False, na_rep='NA'
    )
    summary_stats = pheno_df.groupby('DX_bl').agg({
        'age':['count','mean','std'],
        'sex':'mean',
        'education':['mean','std'],
        'apoe4_count':['mean','std']
    }).round(2)
    summary_stats.to_csv(
        OUTPUT_DIR/"ADNI_AD_vs_CN_summary_stats.txt", sep='\t'
    )

    print(f"\n✅ Files saved to: {OUTPUT_DIR}")
    print(f"\n🏁 Pheno & covar creation complete.")

if __name__ == "__main__":
    main()


In [ ]:
# 01_Data_Prep_QC
# vi. Merge covars with PCA data
# Python script


##############################################################
# USER-DEFINED INPUTS (EDIT HERE) #
##############################################################

covar_file = r"C:\Users\B00731414\OneDrive - Ulster University\6. Code\ADNI\4. Gene Burden\01_data_prep_qc_ADNI1\outputs\ADNI_AD_vs_CN_covariates.txt"

pca_file = r"C:\Users\B00731414\OneDrive - Ulster University\6. Code\ADNI\4. Gene Burden\01_data_prep_qc_ADNI1\outputs\ADNI1_PCA.eigenvec"

output_file = r"C:\Users\B00731414\OneDrive - Ulster University\6. Code\ADNI\4. Gene Burden\01_data_prep_qc_ADNI1\outputs\ADNI1_PCA_merged_covariates.txt"

# Number of PCs to include
num_pcs = 10

##############################################################
# DO NOT EDIT BELOW THIS LINE #
##############################################################

import pandas as pd
import os
from pathlib import Path
from typing import List, Optional

# ── Check input files ───────────────

def validate_input_files(covar_path: str, pca_path: str) -> None:
    print("🔍 Checking input files...")

    if not os.path.exists(covar_path):
        raise FileNotFoundError(f"❌ Covariates file not found: {covar_path}")

    if not os.path.exists(pca_path):
        raise FileNotFoundError(f"❌ PCA file not found: {pca_path}")

    print(f"✅ Covariates file found: {Path(covar_path).name}")
    print(f"✅ PCA file found: {Path(pca_path).name}")

# ── Load PCA scores  ───────────────

def load_pca_data(pca_path: str, num_pcs: int) -> pd.DataFrame:
    print(f"📖 Loading PCA data with {num_pcs} components...")

    try:
        # Define column names for PCA file
        pca_columns = ["FID", "IID"] + [f"PC{i}" for i in range(1, num_pcs + 1)]

        pca = pd.read_csv(
            pca_path,
            sep=r'\s+',
            header=0,
            names=pca_columns,
            dtype={"FID": str, "IID": str}
        )

        print(f"✅ Loaded {len(pca)} samples with {num_pcs} PCs")
        return pca

    except Exception as e:
        raise RuntimeError(f"❌ Error loading PCA data: {e}")

# ── Load covar data w/ str IDs ───────────────

def load_covariate_data(covar_path: str) -> pd.DataFrame:
    print("📖 Loading covariate data...")

    try:
        cov = pd.read_csv(covar_path, sep='\t', dtype=str)

        # Validate required columns
        if 'FID' not in cov.columns or 'IID' not in cov.columns:
            raise ValueError("Covariate file must contain 'FID' and 'IID' columns")

        print(f"✅ Loaded {len(cov)} samples with {len(cov.columns)} covariates")
        print(f"   Covariate columns: {', '.join(cov.columns.tolist())}")
        return cov

    except Exception as e:
        raise RuntimeError(f"❌ Error loading covariate data: {e}")

# ── Merge covar and PCA ───────────────

def merge_data(cov: pd.DataFrame, pca: pd.DataFrame, num_pcs: int) -> pd.DataFrame:
    print("🔗 Merging covariate and PCA data...")

    # Check for sample overlap
    common_samples = set(cov['IID']) & set(pca['IID'])
    cov_only = set(cov['IID']) - set(pca['IID'])
    pca_only = set(pca['IID']) - set(cov['IID'])

    print(f"   Common samples: {len(common_samples)}")
    if cov_only:
        print(f"   ⚠️  Samples in covariates only: {len(cov_only)}")
    if pca_only:
        print(f"   ⚠️  Samples in PCA only: {len(pca_only)}")

    # Merge on IID, keeping covariate FID and dropping PCA FID
    try:
        df = pd.merge(cov, pca.drop(columns="FID"), on="IID", how="inner")

        # Reorder columns: covariate columns first, then PCs
        cov_cols = cov.columns.tolist()
        pc_cols = [f"PC{i}" for i in range(1, num_pcs + 1)]
        df = df[cov_cols + pc_cols]

        print(f"✅ Successfully merged data: {len(df)} samples")
        return df

    except Exception as e:
        raise RuntimeError(f"❌ Error merging data: {e}")

# ── Save merged data ───────────────

def save_merged_data(df: pd.DataFrame, output_path: str) -> None:
    print("💾 Saving merged covariate-PCA file...")

    try:
        # Create output directory if it doesn't exist
        output_dir = Path(output_path).parent
        output_dir.mkdir(parents=True, exist_ok=True)

        # Save with tab delimiter
        df.to_csv(output_path, sep='\t', index=False)

        print(f"✅ Merged file saved: {Path(output_path).name}")
        print(f"   Location: {output_path}")
        print(f"   Dimensions: {df.shape[0]} rows × {df.shape[1]} columns")

    except Exception as e:
        raise RuntimeError(f"❌ Error saving merged data: {e}")

# ── Check data ───────────────
def validate_output(df: pd.DataFrame, num_pcs: int) -> None:

    print("🔍 Checking merged data...")

    # Check for required columns
    required_cols = ['FID', 'IID'] + [f"PC{i}" for i in range(1, num_pcs + 1)]
    missing_cols = [col for col in required_cols if col not in df.columns]

    if missing_cols:
        print(f"⚠️  Warning: Missing expected columns: {missing_cols}")

    # Check for missing values in key columns
    na_counts = df[['FID', 'IID']].isna().sum()
    if na_counts.any():
        print(f"⚠️  Warning: Missing values in ID columns: {na_counts.to_dict()}")

    # Display sample of merged data
    print("\n📋 Sample of merged data:")
    print(df.head(3).to_string(index=False))

    print("✅ Data check complete")

# ── Main function ───────────────

def main():
    print("=" * 60)
    print("COVARIATE-PCA DATA MERGER")
    print("=" * 60)

    try:
        # Validate input files
        validate_input_files(covar_file, pca_file)

        # Load data
        pca_data = load_pca_data(pca_file, num_pcs)
        covar_data = load_covariate_data(covar_file)

        # Merge data
        merged_data = merge_data(covar_data, pca_data, num_pcs)

        # Save merged data
        save_merged_data(merged_data, output_file)

        # Validate output
        validate_output(merged_data, num_pcs)

        print("\n" + "=" * 60)
        print("🏁 MERGE COMPLETE")
        print("=" * 60)
        print(f"✅ Successfully created merged covariate-PCA file")
        print(f"✅ Ready for REGENIE analysis")

    except Exception as e:
        print(f"\n❌ ERROR: {e}")
        print("Please check your input files and try again.")
        exit(1)

if __name__ == "__main__":
    main()

In [ ]:
%%bash

# 01_Data_Prep_QC
# vii. Convert to vcf

##############################################################
# USER-DEFINED INPUTS (EDIT HERE) #
##############################################################

INPUT="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/4. Gene Burden/01_data_prep_qc_ADNI1/outputs/ADNI1_preQC"

OUTPUT="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/4. Gene Burden/02_annotations_ADNI1/inputs/vcf/ADNI1_preQC_converted"

##############################################################
# DO NOT EDIT BELOW THIS LINE #
##############################################################

plink \
    --bfile "${INPUT}" \
    --recode vcf \
    --out "${OUTPUT}"
STATUS=$?

if [ $STATUS -ne 0 ]; then
  echo "❌ ERROR: vcf recode failed with exit code $STATUS"
  exit $STATUS
fi

echo "✅ Files saved: ${OUTPUT}"
echo "🏁 VCF conversion complete"


### 2. Annotation

In [ ]:
%%bash

# 02_annotation
# i. Ensembl Gene Annotation with ANNOVAR

##############################################################
# USER-DEFINED INPUTS (EDIT HERE) #
##############################################################

BASE_DIR="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/4. Gene Burden/02_Annotations_ADNI1"

INPUT_VCF="./inputs/vcf/ADNI1_preQC_converted.vcf"

BUILD="hg18"

OUTPUT_DIR="./outputs"

ANNOVAR_DB_DIR="./inputs/annovar_db"

ANNOVAR_SCRIPTS_DIR="./inputs/annovar_db/annovar"
ANNOVAR_PATH="${ANNOVAR_SCRIPTS_DIR}/table_annovar.pl"
ANNOTATE_VAR_PATH="${ANNOVAR_SCRIPTS_DIR}/annotate_variation.pl"

# Extract project name from input VCF for consistent naming
PROJECT_NAME=$(basename "$INPUT_VCF" | sed 's/_preQC_converted\.vcf$//')

##############################################################
# DO NOT EDIT BELOW THIS LINE #
##############################################################

set -euo pipefail

# ── Change to base directory ───────────────────────────────
cd "$BASE_DIR" || {
    echo "❌ ERROR: Failed to change directory to $BASE_DIR"
    exit 1
}

echo "🔧 Checking ANNOVAR setup..."
echo "📋 Build version: $BUILD"
echo "📁 Project: $PROJECT_NAME"

# ── Verify required scripts exist ──────────────────────────
if [ ! -f "$ANNOTATE_VAR_PATH" ]; then
    echo "❌ ERROR: annotate_variation.pl not found at $ANNOTATE_VAR_PATH"
    echo "Please ensure ANNOVAR scripts are in the correct location"
    exit 1
fi

if [ ! -f "$ANNOVAR_PATH" ]; then
    echo "❌ ERROR: table_annovar.pl not found at $ANNOVAR_PATH"
    exit 1
fi

# ── Check and download ensGene database if needed ──────────
ENSEMBL_DB_FILE="${ANNOVAR_DB_DIR}/${BUILD}_ensGene.txt"

if [ ! -f "$ENSEMBL_DB_FILE" ]; then
    echo "❌ ERROR: ${BUILD}_ensGene.txt database not found"
    echo "🔄 Attempting to download ensGene database for $BUILD..."

    # Download ensGene database using script paths
    perl "$ANNOTATE_VAR_PATH" \
        -buildver "$BUILD" \
        -downdb -webfrom annovar ensGene \
        "$ANNOVAR_DB_DIR/"

    if [ ! -f "$ENSEMBL_DB_FILE" ]; then
        echo "❌ ERROR: Failed to download ensGene database for $BUILD"
        echo "💡 Available builds may be limited. Check ANNOVAR documentation."
        exit 1
    fi
    echo "✅ ensGene database for $BUILD downloaded successfully"
else
    echo "✅ ensGene database for $BUILD found"
fi

# ── Make scripts executable ─────────────────────────────────
chmod +x "$ANNOVAR_SCRIPTS_DIR"/*.pl

# ── Create output directory ─────────────────────────────────
mkdir -p "$OUTPUT_DIR"

echo "🔄 Setting up Ensembl gene annotations..."
echo "⏳ Annotating VCF: $INPUT_VCF with Ensembl genes (Build: $BUILD)..."

# ── Run annotation ──────────────────────────────────────
OUTPUT_PREFIX="${OUTPUT_DIR}/${PROJECT_NAME}_annotated_ensembl"

perl "$ANNOVAR_PATH" \
    "$INPUT_VCF" \
    "$ANNOVAR_DB_DIR/" \
    -buildver "$BUILD" \
    -out "$OUTPUT_PREFIX" \
    -protocol ensGene \
    -operation g \
    -remove \
    -vcfinput \
    -nastring .

# ── Verify annotation success ───────────────────────────────
if [ $? -eq 0 ]; then
    echo "🏁 Annotation finished successfully!"
    echo "📄 Output files:"
    echo "   - ${OUTPUT_PREFIX}.${BUILD}_multianno.txt"
    echo "   - ${OUTPUT_PREFIX}.${BUILD}_multianno.vcf"

    # List actual output files created
    echo ""
    echo "📁 Created files:"
    ls -la "${OUTPUT_PREFIX}".${BUILD}_multianno.*
else
    echo "❌ ERROR: Annotation failed"
    exit 1
fi


In [ ]:
# 02_annotation
# ii. convert ANNOVAR ensGene output to regenie inputs:
# 1. Variant annotations file (.annotations.txt)
# 2. Gene sets file (.sets.txt)
# 3. Mask definitions file (.masks)
# makes file in format  1:69134:A:G	OR4F5(ENSG00000186092) missense(0/5)
# python script

##############################################################
#               USER-DEFINED INPUTS (EDIT HERE)              #
##############################################################

# Path to ANNOVAR ensGene multianno.txt output file
ANNOVAR_FILE = r"C:\Users\B00731414\OneDrive - Ulster University\6. Code\ADNI\4. Gene Burden\02_Annotations_ADNI1\outputs\ADNI1_annotated_ensembl.hg18_multianno.txt"

# Output file prefix for regenie files
OUTPUT_PREFIX = r"C:\Users\B00731414\OneDrive - Ulster University\6. Code\ADNI\4. Gene Burden\02_Annotations_ADNI1\outputs\regenie\regenie_ukb"

# Genome build version
GENOME_BUILD = "hg18"

##############################################################
#            DO NOT EDIT BELOW THIS LINE                     #
##############################################################

import pandas as pd
import numpy as np
from pathlib import Path
import sys
from collections import defaultdict
import re

# ── Parse annotation to extract gene ENSGID ───────────────
def parse_ensembl_gene_name(gene_string):

    if pd.isna(gene_string) or gene_string == '.' or gene_string == '':
        return None

    gene_string = str(gene_string).strip()

    # Split by comma to handle "ENSG00000ID,GENENAME" format
    parts = gene_string.split(',')

    ensg_id = None
    gene_name = None

    # Extract ENSG ID and gene name from parts
    for part in parts:
        part = part.strip()
        if part.startswith('ENSG'):
            ensg_id = part
        elif part and not part.startswith('ENSG'):
            gene_name = part

    # Format like UKB
    if gene_name and ensg_id:
        return f"{gene_name}({ensg_id})"
    elif gene_name:
        return gene_name  # Return gene name even without ENSG
    elif ensg_id:
        return f"Unknown({ensg_id})"  # Fallback if only ENSG available
    else:
        return None

# ── Map ANNOVAR functional annotations to UKB regenie categories ───────────────

def map_annovar_consequence_ukb(func_ensgene, exonic_func_ensgene):

    func = str(func_ensgene).lower()
    exonic_func = str(exonic_func_ensgene).lower()

    # Loss-of-function variants (highest priority)
    if ((func == 'exonic' and any(x in exonic_func for x in
         ['nonsense', 'frameshift', 'stopgain', 'stoploss'])) or
        'splicing' in func):
        return 'LoF'

    # Missense variants
    if func == 'exonic' and any(x in exonic_func for x in
        ['nonsynonymous', 'missense']):
        return 'missense'

    # Synonymous variants
    if func == 'exonic' and 'synonymous' in exonic_func:
        return 'synonymous'

    # Other coding variants
    if func == 'exonic':
        return 'other_coding'

    # Non-coding variants (default)
    return 'non_coding'

# ── check inputs exist ───────────────
def validate_input_file(file_path):

    if not Path(file_path).exists():
        raise FileNotFoundError(f"Input file not found: {file_path}")

    # Test read first few lines to check format
    try:
        df_test = pd.read_csv(file_path, sep='\t', nrows=5, low_memory=False)
        required_cols = ['Gene.ensGene', 'Func.ensGene', 'Chr', 'Start', 'Ref', 'Alt']
        missing_cols = [col for col in required_cols if col not in df_test.columns]

        if missing_cols:
            raise ValueError(f"Missing required columns: {missing_cols}")

        return True
    except Exception as e:
        raise ValueError(f"Error reading input file: {e}")

# ── create variant-gene pairs ───────────────

def create_variant_annotations(df):

    annotations = []
    stats = {'genes_with_ensg': 0, 'genes_without_ensg': 0}

    for _, row in df.iterrows():
        # Create variant ID in regenie format: CHR:POS:REF:ALT
        var_id = f"{row['Chr']}:{row['Start']}:{row['Ref']}:{row['Alt']}"

        # Get gene names from ensGene (may be semicolon-separated)
        genes = str(row['Gene.ensGene'])
        if genes == '.' or pd.isna(genes):
            continue

        # Get functional consequence
        consequence = map_annovar_consequence_ukb(
            row['Func.ensGene'],
            row.get('ExonicFunc.ensGene', '.')
        )

        # Handle multiple genes (separated by semicolons)
        for gene_entry in genes.split(';'):
            gene_entry = gene_entry.strip()
            if gene_entry and gene_entry != '.':
                # Parse and format gene name properly
                formatted_gene = parse_ensembl_gene_name(gene_entry)

                if formatted_gene:
                    annotations.append({
                        'variant_id': var_id,
                        'gene': formatted_gene,
                        'consequence': consequence,
                        'chr': row['Chr'],
                        'pos': row['Start']
                    })

                    # Track ENSG coverage
                    if 'ENSG' in formatted_gene:
                        stats['genes_with_ensg'] += 1
                    else:
                        stats['genes_without_ensg'] += 1

    return pd.DataFrame(annotations), stats

# ── save variant annotations in UKB/regenie format ───────────────

def save_annotations_file(ann_df, output_path):
    with open(output_path, 'w') as f:
        for _, r in ann_df.iterrows():
            f.write(f"{r['variant_id']}\t{r['gene']}\t{r['consequence']}\n")

# ── group variants by gene to create gene sets file ───────────────

def create_gene_sets(ann_df):
    gene_sets = defaultdict(list)
    gene_pos = {}

    for _, r in ann_df.iterrows():
        gene_sets[r['gene']].append(r['variant_id'])
        if r['gene'] not in gene_pos:
            gene_pos[r['gene']] = {'chr': r['chr'], 'pos': r['pos']}

    return gene_sets, gene_pos

# ── save gene sets file ───────────────

def save_gene_sets_file(gene_sets, gene_pos, output_path):
    with open(output_path, 'w') as f:
        for gene, variants in gene_sets.items():
            chrpos = gene_pos[gene]
            if variants:  # Only include genes with variants
                f.write(f"{gene} {chrpos['chr']} {chrpos['pos']} " +
                       f"{','.join(variants)}\n")


# ── create masks ───────────────
def create_mask_definitions():
    return {
        'M1': ['LoF'],  # Loss-of-function only
        'M2': ['missense'],  # Missense variants only
        'M3': ['LoF', 'missense'],  # Combined LoF and missense
        'M4': ['LoF', 'missense', 'synonymous'],  # All coding variants
    }

# ── save masks ───────────────
def save_mask_definitions_file(mask_definitions, output_path):

    with open(output_path, 'w') as f:
        for mask_name, consequences in mask_definitions.items():
            f.write(f"{mask_name} {','.join(consequences)}\n")

# ── main function  ───────────────
def create_regenie_files_from_annovar():
    print("=" * 80)
    print("ANNOVAR ENSEMBL TO REGENIE CONVERSION")
    print("=" * 80)
    print(f"Input file: {ANNOVAR_FILE}")
    print(f"Output prefix: {OUTPUT_PREFIX}")
    print(f"Genome build: {GENOME_BUILD}")
    print()

    try:
        # Validate input file
        validate_input_file(ANNOVAR_FILE)

        # Read ANNOVAR ensGene output
        print("📖 Reading ANNOVAR ensGene file...")
        df = pd.read_csv(ANNOVAR_FILE, sep='\t', low_memory=False)
        print(f"✅ Found {len(df)} annotated variants")

        # Create variant annotations
        print("\n🧬 Creating variant annotations...")
        ann_df, stats = create_variant_annotations(df)

        if len(ann_df) == 0:
            raise ValueError("No valid annotations created")

        print(f"✅ Created {len(ann_df)} variant-gene annotation pairs")
        print(f"✅ {stats['genes_with_ensg']} annotations have ENSG IDs")
        print(f"⚠️  {stats['genes_without_ensg']} annotations lack ENSG IDs")

        # Show example annotations
        if len(ann_df) > 0:
            print("\n📋 Example annotations:")
            for i in range(min(5, len(ann_df))):
                row = ann_df.iloc[i]
                print(f"   {row['variant_id']} {row['gene']} {row['consequence']}")

        # Save annotations file
        ann_file = f"{OUTPUT_PREFIX}.annotations.txt"
        save_annotations_file(ann_df, ann_file)
        print(f"💾 Saved: {ann_file}")

        # Create and save gene sets
        print("\n🧮 Creating gene sets...")
        gene_sets, gene_pos = create_gene_sets(ann_df)

        sets_file = f"{OUTPUT_PREFIX}.sets.txt"
        save_gene_sets_file(gene_sets, gene_pos, sets_file)
        print(f"✅ Created {len(gene_sets)} gene sets")
        print(f"💾 Saved: {sets_file}")

        # Create and save mask definitions
        print("\n🎭 Creating mask definitions...")
        mask_definitions = create_mask_definitions()

        masks_file = f"{OUTPUT_PREFIX}.masks"
        save_mask_definitions_file(mask_definitions, masks_file)
        print(f"✅ Created {len(mask_definitions)} mask definitions")
        print(f"💾 Saved: {masks_file}")

        # Final summary
        print("\n" + "=" * 80)
        print("CONVERSION COMPLETE")
        print("=" * 80)
        print(f"✅ Processed {len(df)} variants")
        print(f"✅ Created annotations for {len(gene_sets)} genes")
        print(f"✅ Generated {len(mask_definitions)} mask types")
        print(f"✅ Gene name format: GENENAME(ENSG00000ID)")

        if len(ann_df) > 0:
            example_row = ann_df.iloc[0]
            print(f"   Example: {example_row['variant_id']}\t{example_row['gene']}\t{example_row['consequence']}")

    except Exception as e:
        print(f"❌ ERROR: {e}")
        sys.exit(1)

if __name__ == "__main__":
   create_regenie_files_from_annovar()

In [ ]:
# 02_annotation
# ii. (ALTERNATIVE) convert ANNOVAR ensGene output to regenie inputs:
# 1. Variant annotations file (.annotations.txt)
# 2. Gene sets file (.sets.txt)
# 3. Mask definitions file (.masks)
# makes file in format: rs17160698 NOC2L(ENSG00000ID) non_coding
# python script


##############################################################
#               USER-DEFINED INPUTS (EDIT HERE)              #
##############################################################

# Path to ANNOVAR ensGene multianno.txt output file
ANNOVAR_FILE = r"C:\Users\B00731414\OneDrive - Ulster University\6. Code\ADNI\4. Gene Burden\02_Annotations_ADNI1\outputs\ADNI1_annotated_ensembl.hg18_multianno.txt"

# Path to the BIM file to get the correct variant IDs (rsIDs)
BIM_FILE = r"C:\Users\B00731414\OneDrive - Ulster University\6. Code\ADNI\4. Gene Burden\01_data_prep_qc_ADNI1\outputs\ADNI1_preQC.bim"

# Output file prefix for regenie files
OUTPUT_PREFIX = r"C:\Users\B00731414\OneDrive - Ulster University\6. Code\ADNI\4. Gene Burden\02_Annotations_ADNI1\outputs\regenie\regenie_ukb"

# Genome build version
GENOME_BUILD = "hg18"

##############################################################
#            DO NOT EDIT BELOW THIS LINE                     #
##############################################################

import pandas as pd
import numpy as np
from pathlib import Path
import sys
from collections import defaultdict
import re

# ── Parse annotation to extract gene ENSGID ───────────────
def parse_ensembl_gene_name(gene_string):
    if pd.isna(gene_string) or gene_string == '.' or gene_string == '':
        return None

    gene_string = str(gene_string).strip()
    parts = gene_string.split(',')

    ensg_id = None
    gene_name = None

    for part in parts:
        part = part.strip()
        if part.startswith('ENSG'):
            ensg_id = part
        elif part and not part.startswith('ENSG'):
            gene_name = part

    if gene_name and ensg_id:
        return f"{gene_name}({ensg_id})"
    elif gene_name:
        return gene_name
    elif ensg_id:
        return f"Unknown({ensg_id})"
    else:
        return None

# ── Map ANNOVAR functional annotations to UKB regenie categories ───────────────
def map_annovar_consequence_ukb(func_ensgene, exonic_func_ensgene):
    func = str(func_ensgene).lower()
    exonic_func = str(exonic_func_ensgene).lower()

    if ((func == 'exonic' and any(x in exonic_func for x in
         ['nonsense', 'frameshift', 'stopgain', 'stoploss'])) or
        'splicing' in func):
        return 'LoF'

    if func == 'exonic' and any(x in exonic_func for x in
        ['nonsynonymous', 'missense']):
        return 'missense'

    if func == 'exonic' and 'synonymous' in exonic_func:
        return 'synonymous'

    if func == 'exonic':
        return 'other_coding'

    return 'non_coding'

# ── Load BIM file and create CHR:POS:REF:ALT → rsID mapping ───────────────
def load_bim_file(bim_path):
    print("📖 Reading BIM file to create variant ID mapping...")

    if not Path(bim_path).exists():
        raise FileNotFoundError(f"BIM file not found: {bim_path}")

    bim_df = pd.read_csv(bim_path, sep=r'\s+', header=None,
                         names=['CHR', 'rsID', 'cM', 'POS', 'A1', 'A2'])

    print(f"✅ Loaded {len(bim_df)} variants from BIM file")

    variant_to_rsid = {}
    for _, row in bim_df.iterrows():
        var_id_1 = f"{row['CHR']}:{row['POS']}:{row['A1']}:{row['A2']}"
        var_id_2 = f"{row['CHR']}:{row['POS']}:{row['A2']}:{row['A1']}"
        variant_to_rsid[var_id_1] = row['rsID']
        variant_to_rsid[var_id_2] = row['rsID']

    print(f"✅ Created mapping for {len(variant_to_rsid)} variant ID combinations")
    return variant_to_rsid

# ── Validate ANNOVAR input ───────────────
def validate_annovar_file(file_path):
    if not Path(file_path).exists():
        raise FileNotFoundError(f"ANNOVAR file not found: {file_path}")

    df_test = pd.read_csv(file_path, sep='\t', nrows=5, low_memory=False)
    required_cols = ['Gene.ensGene', 'Func.ensGene', 'Chr', 'Start', 'Ref', 'Alt']
    missing_cols = [col for col in required_cols if col not in df_test.columns]

    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    return True

# ── Create variant annotations with rsID mapping ───────────────
def create_variant_annotations(df, variant_to_rsid):
    annotations = []
    stats = {
        'genes_with_ensg': 0,
        'genes_without_ensg': 0,
        'variants_mapped': 0,
        'variants_unmapped': 0
    }

    for _, row in df.iterrows():
        chr_pos_ref_alt = f"{row['Chr']}:{row['Start']}:{row['Ref']}:{row['Alt']}"

        if chr_pos_ref_alt in variant_to_rsid:
            rsid = variant_to_rsid[chr_pos_ref_alt]
            stats['variants_mapped'] += 1
        else:
            chr_pos_alt_ref = f"{row['Chr']}:{row['Start']}:{row['Alt']}:{row['Ref']}"
            if chr_pos_alt_ref in variant_to_rsid:
                rsid = variant_to_rsid[chr_pos_alt_ref]
                stats['variants_mapped'] += 1
            else:
                stats['variants_unmapped'] += 1
                continue

        genes = str(row['Gene.ensGene'])
        if genes == '.' or pd.isna(genes):
            continue

        consequence = map_annovar_consequence_ukb(
            row['Func.ensGene'],
            row.get('ExonicFunc.ensGene', '.')
        )

        for gene_entry in genes.split(';'):
            gene_entry = gene_entry.strip()
            if gene_entry and gene_entry != '.':
                formatted_gene = parse_ensembl_gene_name(gene_entry)
                if formatted_gene:
                    annotations.append({
                        'variant_id': rsid,
                        'gene': formatted_gene,
                        'consequence': consequence,
                        'chr': row['Chr'],
                        'pos': row['Start']
                    })
                    if 'ENSG' in formatted_gene:
                        stats['genes_with_ensg'] += 1
                    else:
                        stats['genes_without_ensg'] += 1

    return pd.DataFrame(annotations), stats

# ── Save annotations file ───────────────
def save_annotations_file(ann_df, output_path):
    with open(output_path, 'w') as f:
        for _, r in ann_df.iterrows():
            f.write(f"{r['variant_id']}\t{r['gene']}\t{r['consequence']}\n")

# ── Create gene sets ───────────────
def create_gene_sets(ann_df):
    gene_sets = defaultdict(list)
    gene_pos = {}
    for _, r in ann_df.iterrows():
        gene_sets[r['gene']].append(r['variant_id'])
        if r['gene'] not in gene_pos:
            gene_pos[r['gene']] = {'chr': r['chr'], 'pos': r['pos']}
    return gene_sets, gene_pos

# ── Save gene sets file ───────────────
def save_gene_sets_file(gene_sets, gene_pos, output_path):
    with open(output_path, 'w') as f:
        for gene, variants in gene_sets.items():
            chrpos = gene_pos[gene]
            if variants:
                f.write(f"{gene} {chrpos['chr']} {chrpos['pos']} " +
                        f"{','.join(variants)}\n")

# ── Create mask definitions ───────────────
def create_mask_definitions():
    return {
        'M1': ['LoF'],
        'M2': ['missense'],
        'M3': ['LoF', 'missense'],
        'M4': ['LoF', 'missense', 'synonymous'],
    }

# ── Save mask definitions ───────────────
def save_mask_definitions_file(mask_definitions, output_path):
    with open(output_path, 'w') as f:
        for mask_name, consequences in mask_definitions.items():
            f.write(f"{mask_name} {','.join(consequences)}\n")

# ── Main function ───────────────
def create_regenie_files_with_rsid_mapping():
    print("=" * 80)
    print("ANNOVAR ENSEMBL TO REGENIE CONVERSION (rsID mapping)")
    print("=" * 80)
    print(f"Input ANNOVAR file: {ANNOVAR_FILE}")
    print(f"Input BIM file: {BIM_FILE}")
    print(f"Output prefix: {OUTPUT_PREFIX}")
    print(f"Genome build: {GENOME_BUILD}")
    print()

    try:
        validate_annovar_file(ANNOVAR_FILE)
        variant_to_rsid = load_bim_file(BIM_FILE)

        print("📖 Reading ANNOVAR ensGene file...")
        df = pd.read_csv(ANNOVAR_FILE, sep='\t', low_memory=False)
        print(f"✅ Found {len(df)} annotated variants")

        print("\n🧬 Creating variant annotations with rsID mapping...")
        ann_df, stats = create_variant_annotations(df, variant_to_rsid)

        if len(ann_df) == 0:
            raise ValueError("No valid annotations created")

        print(f"✅ Created {len(ann_df)} variant-gene annotation pairs")
        print(f"✅ {stats['variants_mapped']} variants successfully mapped to rsIDs")
        print(f"⚠️  {stats['variants_unmapped']} variants could not be mapped")
        print(f"✅ {stats['genes_with_ensg']} annotations have ENSG IDs")
        print(f"⚠️  {stats['genes_without_ensg']} annotations lack ENSG IDs")

        if len(ann_df) > 0:
            print("\n📋 Example annotations:")
            for i in range(min(5, len(ann_df))):
                row = ann_df.iloc[i]
                print(f"   {row['variant_id']} {row['gene']} {row['consequence']}")

        ann_file = f"{OUTPUT_PREFIX}.annotations.txt"
        save_annotations_file(ann_df, ann_file)
        print(f"💾 Saved: {ann_file}")

        print("\n🧮 Creating gene sets...")
        gene_sets, gene_pos = create_gene_sets(ann_df)
        sets_file = f"{OUTPUT_PREFIX}.sets.txt"
        save_gene_sets_file(gene_sets, gene_pos, sets_file)
        print(f"✅ Created {len(gene_sets)} gene sets")
        print(f"💾 Saved: {sets_file}")

        print("\n🎭 Creating mask definitions...")
        mask_definitions = create_mask_definitions()
        masks_file = f"{OUTPUT_PREFIX}.masks"
        save_mask_definitions_file(mask_definitions, masks_file)
        print(f"✅ Created {len(mask_definitions)} mask definitions")
        print(f"💾 Saved: {masks_file}")

        print("\n" + "=" * 80)
        print("CONVERSION COMPLETE (rsID mapping)")
        print("=" * 80)
        print(f"✅ Processed {len(df)} ANNOVAR variants")
        print(f"✅ Mapped {stats['variants_mapped']} variants to rsIDs")
        print(f"⚠️  {stats['variants_unmapped']} variants unmapped")
        print(f"✅ Created annotations for {len(gene_sets)} genes")
        print(f"✅ Generated {len(mask_definitions)} mask types")

        if len(ann_df) > 0:
            example_row = ann_df.iloc[0]
            print(f"   Example: {example_row['variant_id']}\t{example_row['gene']}\t{example_row['consequence']}")

    except Exception as e:
        print(f"❌ ERROR: {e}")
        sys.exit(1)

if __name__ == "__main__":
    create_regenie_files_with_rsid_mapping()


### 3. Regenie

In [ ]:
%%bash

# 03_regenie
# step 1
# create null model to control for relatedness, population structure and polygenicity.

##############################################################
#               USER-DEFINED INPUTS (EDIT HERE)              #
##############################################################

BED_FILE="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/4. Gene Burden/01_Data_Prep_QC/outputs/ADNI2_preQC_common"
PHENO_FILE="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/4. Gene Burden/01_Data_Prep_QC/outputs/ADNI_AD_vs_CN_phenotypes.txt"
PHENO_COL="AD_vs_CN"
COVAR_FILE="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/4. Gene Burden/01_Data_Prep_QC/outputs/ADNI_PCA_merged_covariates.txt"
COVAR_COL_LIST="age,sex,education,apoe4_count,PC{1:10}"
OUTPUT_PREFIX="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/4. Gene Burden/03_Regenie/step 1/S1_ADNI"

##############################################################
#            DO NOT EDIT BELOW THIS LINE                     #
##############################################################

regenie \
  --step 1 \
  --bt \ # for binary trait
  --ref-first \
  --bed "$BED_FILE" \
  --phenoFile "$PHENO_FILE" \
  --phenoCol $PHENO_COL \
  --covarFile "$COVAR_FILE" \
  --covarColList $COVAR_COL_LIST \
  --bsize 200 \
  --gz \
  --out "$OUTPUT_PREFIX"


In [ ]:
#!/bin/bash

#SBATCH --job-name=regenie_step1
#SBATCH --output=/users/smitchell/ADNI/ADNI1_regenie/logs_and_errors/regenie_step1_%j.out
#SBATCH --error=/users/smitchell/ADNI/ADNI1_regenie/logs_and_errors/regenie_step1_%j.err
#SBATCH --time=02:00:00
#SBATCH --partition=k2-hipri
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=4
#SBATCH --mem=64G
#SBATCH --mail-type=BEGIN,END,FAIL
#SBATCH --mail-user=mitchell-s17@ulster.ac.uk

##############################################################
#               USER-DEFINED INPUTS (EDIT HERE)              #
##############################################################

# Input directories
INPUT_PLINK_PREFIX="/users/smitchell/ADNI/ADNI1_regenie/pre_qc/ADNI1_preQC_common"
PHENO_FILE="/users/smitchell/ADNI/ADNI1_regenie/pre_qc/ADNI_AD_vs_CN_phenotypes.txt"
COVAR_FILE="/users/smitchell/ADNI/ADNI1_regenie/pre_qc/ADNI1_PCA_merged_covariates.txt"

# Output directory for step 1
OUT_DIR="/users/smitchell/ADNI/ADNI_regenie/regenie_step_1"

##############################################################
#            DO NOT EDIT BELOW THIS LINE                     #
##############################################################

# ── activate conda env ───────────────────────────────────────────

# Initialise conda (adjust path if needed)
eval "$(conda shell.bash hook)"

# Activate the regenie environment
conda activate regenie-env


# ── load modules ───────────────────────────────────────────

module load compilers/gcc/14.1.0
module load libs/intel-mkl/2020u4/bin

# ── Run REGENIE step 1 ───────────────────────────────────────────

regenie \
  --step 1 \
  --bt \
  --ref-first \
  --bed "${INPUT_PLINK_PREFIX}" \
  --phenoFile "${PHENO_FILE}" \
  --phenoCol AD_vs_CN \
  --covarFile "${COVAR_FILE}" \
  --covarColList age,sex,education,apoe4_count,PC{1:10} \
  --bsize 200 \
  --gz \
  --out "${OUT_DIR}/S1_ADNI"

# ── deactivate conda ───────────────────────────────────────────
conda deactivate


In [ ]:
%%bash

# 03_regenie
# step 2
# burden testing for rare variants

##############################################################
#               USER-DEFINED INPUTS (EDIT HERE)              #
##############################################################

BASE_DIR="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/4. Gene Burden"
PHENO_FILE="$BASE_DIR/01_Data_Prep_QC/outputs/ADNI_AD_vs_CN_phenotypes.txt"
COVAR_FILE="$BASE_DIR/01_Data_Prep_QC/outputs/ADNI_PCA_merged_covariates.txt"
RARE_BED="$BASE_DIR/01_Data_Prep_QC/outputs/ADNI2_preQC"

# Annotation files
ANNO_FILE="$BASE_DIR/02_Annotations/outputs/regenie/regenie_ukb.annotations.txt"
SETS_FILE="$BASE_DIR/02_Annotations/outputs/regenie/regenie_ukb.sets.txt"
MASK_FILE="$BASE_DIR/02_Annotations/outputs/regenie/regenie_ukb.masks"

# Output directories
STEP1_OUT="$BASE_DIR/03_Regenie/step 1"
STEP2_OUT="$BASE_DIR/03_Regenie/step 2"

##############################################################
#            DO NOT EDIT BELOW THIS LINE                     #
##############################################################

regenie \
  --step 2 \
  --bt \
  --bed "$RARE_BED" \
  --ref-first \
  --phenoFile "$PHENO_FILE" \
  --phenoCol AD_vs_CN \
  --covarFile "$COVAR_FILE" \
  --covarColList age,sex,education,apoe4_count,PC{1:10} \
  --pred "$STEP1_OUT/S1_ADNI_pred_base.list" \
  --anno-file "$ANNO_FILE" \
  --set-list "$SETS_FILE" \
  --mask-def "$MASK_FILE" \
  --aaf-bins 0.01,0.001 \
  --build-mask max \
  --write-mask \
  --vc-tests skato,acato \
  --bsize 200 \
  --out "$STEP2_OUT/S2_ADNI_burden"

In [ ]:
#!/bin/bash
#SBATCH --job-name=regenie_step2
#SBATCH --output=/users/smitchell/ADNI/ADNI1_regenie/logs_and_errors/regenie_step2_%j.out
#SBATCH --error=/users/smitchell/ADNI/ADNI1_regenie/logs_and_errors/regenie_step2_%j.err
#SBATCH --time=2:00:00
#SBATCH --partition=k2-hipri
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=8
#SBATCH --mem=128G
#SBATCH --mail-type=BEGIN,END,FAIL
#SBATCH --mail-user=mitchell-s17@ulster.ac.uk

##############################################################
#               USER-DEFINED INPUTS (EDIT HERE)              #
##############################################################

BASE_DIR="/users/smitchell/ADNI/ADNI1_regenie"

PHENO_FILE="${BASE_DIR}/pre_qc/ADNI_AD_vs_CN_phenotypes.txt"
COVAR_FILE="${BASE_DIR}/pre_qc/ADNI1_PCA_merged_covariates.txt"
RARE_BED="${BASE_DIR}/pre_qc/ADNI1_preQC"

# Annotation files
ANNO_FILE="${BASE_DIR}/annotations/regenie_ukb.annotations.txt"
SETS_FILE="${BASE_DIR}/annotations/regenie_ukb.sets.txt"
MASK_FILE="${BASE_DIR}/annotations/regenie_ukb.masks"

# Output directory for step 2
OUT_DIR="${BASE_DIR}/regenie_step_2"
mkdir -p "${OUT_DIR}"

##############################################################
#            DO NOT EDIT BELOW THIS LINE                     #
##############################################################

# ── activate conda env ───────────────────────────────────────────

eval "$(conda shell.bash hook)"
conda activate regenie-env

# ── load modules ───────────────────────────────────────────

module load compilers/gcc/14.1.0
module load libs/eigen/3.4.0/gcc-14.1.0
module load libs/intel-mkl/2020u4/bin

# ── run regenie step 2 ───────────────────────────────────────────

regenie \
  --step 2 \
  --bt \
  --bed "${RARE_BED}" \
  --ref-first \
  --phenoFile "${PHENO_FILE}" \
  --phenoCol AD_vs_CN \
  --covarFile "${COVAR_FILE}" \
  --covarColList age,sex,education,apoe4_count,PC{1:10} \
  --pred "${BASE_DIR}/regenie_step_1/S1_ADNI_pred.list" \
  --anno-file "${ANNO_FILE}" \
  --set-list "${SETS_FILE}" \
  --mask-def "${MASK_FILE}" \
  --aaf-bins 0.01,0.001 \
  --build-mask max \
  --write-mask \
  --vc-tests skato,acato \
  --bsize 200 \
  --out "${OUT_DIR}/S2_ADNI_burden"

# ── deactivate conda ───────────────────────────────────────────
conda deactivate



In [ ]:
# 04_LOVO_Gene_Selection
# i. Select top genes from REGENIE burden test results (LOVO gene list)
# Python script

##############################################################
#               USER-DEFINED INPUTS (EDIT HERE)              #
##############################################################

# Path to REGENIE output
REGENIE_FILE = r"C:\Users\B00731414\OneDrive - Ulster University\6. Code\ADNI\4. Gene Burden\03_Regenie_ADNI1\S2_ADNI_burden_AD_vs_CN.regenie"

# Test configuration
TEST_NAME = "ADD-SKATO"        # e.g. ADD-SKATO, ADD
TOP_N = 10                     # Number of top gene–mask entries to select
AAF_BIN_LABEL = "all"          # Label for AAF bin (as in ID field)
OUTPUT_FILE = "lovo_genes.txt"

# Scoring method:
# "logp"  = Rank by LOG10P (recommended for burden tests)
# "chisq" = Rank by CHISQ statistic
# "beta"  = Rank by absolute BETA
SCORE_METHOD = "logp"

# Duplicate handling strategy:
# "best_mask"     → Keep most restrictive mask (M1 > M2 > M3 > M4)
# "keep_all"      → Keep all mask combinations
# "highest_score" → Keep highest scoring entry per gene
DUPLICATE_STRATEGY = "keep_all"

# significance filtering
SIGNIFICANCE_THRESHOLD = None  # Set to p-value threshold (e.g., 0.05) or None to disable

##############################################################
#               DO NOT EDIT BELOW THIS LINE                  #
##############################################################

import pandas as pd
import numpy as np
import sys
from pathlib import Path

# ── Compute score ranking ───────────────────────────
def compute_score(df, method):
    df_scored = df.copy()
    if method == "logp":
        df_scored = df_scored[df_scored["LOG10P"] > 0].copy()
        df_scored["score"] = df_scored["LOG10P"]
    elif method == "chisq":
        df_scored = df_scored[df_scored["CHISQ"] > 0].copy()
        df_scored["score"] = df_scored["CHISQ"]
    elif method == "beta":
        df_scored = df_scored[df_scored["BETA"].notna() & (df_scored["BETA"] != 0)].copy()
        if len(df_scored) == 0:
            print("⚠️  WARNING: No valid BETA values found!")
            return df_scored
        df_scored["score"] = df_scored["BETA"].abs()
    else:
        raise ValueError(f"Unknown scoring method: {method}")
    return df_scored

# ── Extract gene/mask info from ID col ────────────────
def extract_gene_mask_info(df, aaf_label):
    id_parts = df["ID"].str.split(".", expand=True)
    if id_parts.shape[1] >= 3:
        df["GENE"] = id_parts[0]
        df["MASK"] = id_parts[1]
        df["AAF_BIN"] = aaf_label
    else:
        print("⚠️  WARNING: ID format may not match expected GENE.MASK.AAF pattern")
        df["GENE"] = df["ID"]
        df["MASK"] = "unknown"
        df["AAF_BIN"] = aaf_label
    return df

# ── Handle duplicates ─────────────────────────────
def filter_duplicates(df, strategy="best_mask"):
    if strategy == "keep_all":
        return df
    elif strategy == "highest_score":
        return df.loc[df.groupby("GENE")["score"].idxmax()].copy()
    elif strategy == "best_mask":
        mask_priority = {"M1": 1, "M2": 2, "M3": 3, "M4": 4}
        df["mask_priority"] = df["MASK"].map(mask_priority).fillna(99)
        best = df.loc[df.groupby("GENE")["mask_priority"].idxmin()].copy()
        return best.drop("mask_priority", axis=1)
    else:
        raise ValueError(f"Unknown duplicate strategy: {strategy}")

# ── Apply significance filtering ─────────────────────────────
def apply_significance_filter(df, threshold):
    if threshold is None:
        return df

    # Convert LOG10P back to p-value for filtering
    if "LOG10P" in df.columns:
        df_filtered = df[df["LOG10P"] >= -np.log10(threshold)].copy()
        print(f"   Applied p < {threshold} filter: {len(df_filtered)} entries remain")
        return df_filtered
    else:
        print("⚠️  WARNING: Cannot apply significance filter without LOG10P column")
        return df

# ── Load regenie results ──────────────────────────────────────────
def main():
    print("\n" + "="*80)
    print("LOVO GENE SELECTION FROM REGENIE BURDEN TEST RESULTS")
    print("="*80)

    # ── validate parameters ──────────────────────────────────────────
    print("\n🔧 Validating parameters...")
    if TOP_N <= 0:
        print(f"❌ ERROR: TOP_N must be positive, got {TOP_N}")
        sys.exit(1)

    regenie_path = Path(REGENIE_FILE)
    if not regenie_path.exists():
        print(f"❌ ERROR: Input file not found: {REGENIE_FILE}")
        sys.exit(1)

    if SCORE_METHOD not in ["logp", "chisq", "beta"]:
        print(f"❌ ERROR: Invalid SCORE_METHOD: {SCORE_METHOD}")
        sys.exit(1)

    if DUPLICATE_STRATEGY not in ["best_mask", "keep_all", "highest_score"]:
        print(f"❌ ERROR: Invalid DUPLICATE_STRATEGY: {DUPLICATE_STRATEGY}")
        sys.exit(1)

    print("✅ All parameters validated")

    # ── load results ──────────────────────────────────────────
    print(f"\n📖 Loading REGENIE results from: {REGENIE_FILE}")
    try:
        df = pd.read_csv(REGENIE_FILE, sep=r"\s+", comment="#", header=0)
        print(f"✅ Loaded {len(df)} rows × {len(df.columns)} columns")
        print(f"   Available tests: {', '.join(df['TEST'].unique())}")
    except Exception as e:
        print(f"❌ ERROR loading file: {e}")
        sys.exit(1)

    # ── filter to test ──────────────────────────────────────────
    print(f"\n🔎 Filtering to test: {TEST_NAME}")
    df_test = df[df["TEST"] == TEST_NAME].copy()
    if df_test.empty:
        print(f"❌ ERROR: No results found for '{TEST_NAME}'")
        sys.exit(1)
    print(f"✅ Found {len(df_test)} entries for {TEST_NAME}")

    # ── extract gene and mask info ──────────────────────────────────────────
    print(f"\n🧬 Extracting gene and mask information...")
    df_test = extract_gene_mask_info(df_test, AAF_BIN_LABEL)
    print(f"✅ Found {df_test['GENE'].nunique()} unique genes")
    print("   Mask distribution:")
    for mask, count in df_test['MASK'].value_counts().items():
        print(f"     {mask}: {count} entries")

    # ── compute scores ──────────────────────────────────────────
    print(f"\n📊 Computing scores (method: {SCORE_METHOD})")
    df_scored = compute_score(df_test, SCORE_METHOD)
    if df_scored.empty:
        print(f"❌ ERROR: No valid scores with method '{SCORE_METHOD}'")
        sys.exit(1)
    print(f"✅ {len(df_scored)} entries with valid scores")
    print(f"   Score range: {df_scored['score'].min():.4f} → {df_scored['score'].max():.4f}")

    # ── apply significance filtering ──────────────────────────────────────────
    if SIGNIFICANCE_THRESHOLD is not None:
        print(f"\n🎯 Applying significance filtering (p < {SIGNIFICANCE_THRESHOLD})...")
        df_scored = apply_significance_filter(df_scored, SIGNIFICANCE_THRESHOLD)
        if df_scored.empty:
            print(f"❌ ERROR: No significant results at p < {SIGNIFICANCE_THRESHOLD}")
            sys.exit(1)

    # ── handle duplicates ──────────────────────────────────────────
    print(f"\n🔄 Handling duplicate genes (strategy: {DUPLICATE_STRATEGY})...")
    initial_count = len(df_scored)
    unique_genes_initial = df_scored['GENE'].nunique()

    df_scored = filter_duplicates(df_scored, strategy=DUPLICATE_STRATEGY)

    final_count = len(df_scored)
    unique_genes_final = df_scored['GENE'].nunique()

    print(f"✅ Deduplication complete:")
    print(f"   Before: {initial_count} entries, {unique_genes_initial} unique genes")
    print(f"   After:  {final_count} entries, {unique_genes_final} unique genes")

    # ── select top entries ──────────────────────────────────────────
    print(f"\n🏆 Selecting top {TOP_N} entries")
    df_top = df_scored.sort_values("score", ascending=False).head(TOP_N)

    print("\nTop gene–mask combinations:")
    print("-" * 80)
    for i, (_, row) in enumerate(df_top.iterrows(), 1):
        score_label = {"logp": "LOG10P", "chisq": "CHISQ", "beta": "|BETA|"}[SCORE_METHOD]
        print(f"{i:2d}. {row['GENE']:<12} ({row['MASK']}) - {score_label}={row['score']:.4f}")

    # ── write outputs ──────────────────────────────────────────
    print(f"\n💾 Writing LOVO gene list → {OUTPUT_FILE}")
    with open(OUTPUT_FILE, "w") as fout:
        # Write header
        fout.write("GENE,MASK,AAF_BIN\n")
        for _, row in df_top.iterrows():
            fout.write(f"{row['GENE']},{row['MASK']},{row['AAF_BIN']}\n")
    print(f"✅ Saved {len(df_top)} entries to {OUTPUT_FILE}")

    # ── summary ──────────────────────────────────────────
    print("\n" + "="*80)
    print("SUMMARY")
    print("="*80)
    print(f"📂 Input file: {REGENIE_FILE}")
    print(f"🧪 Test analysed: {TEST_NAME}")
    print(f"📊 Scoring method: {SCORE_METHOD}")
    print(f"🔄 Duplicate strategy: {DUPLICATE_STRATEGY}")
    print(f"🎯 Significance filter: {SIGNIFICANCE_THRESHOLD if SIGNIFICANCE_THRESHOLD else 'None'}")
    print(f"📈 Total entries analysed: {len(df_scored)}")
    print(f"🏆 Top entries selected: {TOP_N}")
    print(f"💾 Output file: {OUTPUT_FILE}")

    # ── detailed results ──────────────────────────────────────────
    base_cols = ["GENE", "MASK", "AAF_BIN"]
    optional_cols = [c for c in ["BETA", "LOG10P", "CHISQ"] if c in df_top.columns]
    display_cols = base_cols + optional_cols
    print("\n📋 Detailed results:")
    print(df_top[display_cols].to_string(index=False, float_format="%.4f"))

    print("\nLOVO gene list format (GENE,MASK,AAF_BIN):")
    for _, row in df_top.iterrows():
        print(f"{row['GENE']},{row['MASK']},{row['AAF_BIN']}")

if __name__ == "__main__":
    main()

In [ ]:
# version with boniferri correction
# multiple testing correction as testing thousands of genes simultaneously, so can expect false positives


# 04_LOVO_Gene_Selection
# i. Select top genes from REGENIE burden test results (LOVO gene list)
# Python script - IMPROVED VERSION WITH BONFERRONI CORRECTION

##############################################################
#               USER-DEFINED INPUTS (EDIT HERE)              #
##############################################################

# Path to REGENIE output
REGENIE_FILE = r"C:\Users\B00731414\OneDrive - Ulster University\6. Code\ADNI\4. Gene Burden\03_Regenie_ADNI1\S2_ADNI_burden_AD_vs_CN.regenie"

# Test configuration
TEST_NAME = "ADD-SKATO"        # e.g. ADD-SKATO, ADD
TOP_N = 10                     # Number of top gene–mask entries to select
AAF_BIN_LABEL = "all"          # Label for AAF bin (as in ID field)
OUTPUT_FILE = "lovo_genes.txt"

# Scoring method:
# "logp"  = Rank by LOG10P (recommended for burden tests)
# "chisq" = Rank by CHISQ statistic
# "beta"  = Rank by absolute BETA
SCORE_METHOD = "logp"

# duplicate handling strategy:
# "best_mask"     → Keep most restrictive mask (M1 > M2 > M3 > M4)
# "keep_all"      → Keep all mask combinations
# "highest_score" → Keep highest scoring entry per gene
DUPLICATE_STRATEGY = "keep_all"

# multiple testing correction
APPLY_BONFERRONI = True        # Apply Bonferroni correction for multiple testing
BONFERRONI_ALPHA = 0.05        # Family-wise error rate for Bonferroni correction

# Raw significance filtering (applied before Bonferroni if enabled)
RAW_SIGNIFICANCE_THRESHOLD = None  # Set to raw p-value threshold (e.g., 0.05) or None to disable

##############################################################
#               DO NOT EDIT BELOW THIS LINE                  #
##############################################################

import pandas as pd
import numpy as np
import sys
from pathlib import Path

# ── Compute score ranking ───────────────────────────────
def compute_score(df, method):
    df_scored = df.copy()
    if method == "logp":
        df_scored = df_scored[df_scored["LOG10P"] > 0].copy()
        df_scored["score"] = df_scored["LOG10P"]
    elif method == "chisq":
        df_scored = df_scored[df_scored["CHISQ"] > 0].copy()
        df_scored["score"] = df_scored["CHISQ"]
    elif method == "beta":
        df_scored = df_scored[df_scored["BETA"].notna() & (df_scored["BETA"] != 0)].copy()
        if len(df_scored) == 0:
            print("⚠️  WARNING: No valid BETA values found!")
            return df_scored
        df_scored["score"] = df_scored["BETA"].abs()
    else:
        raise ValueError(f"Unknown scoring method: {method}")
    return df_scored

# ── Extract gene/mask info from ID col ────────────────
def extract_gene_mask_info(df, aaf_label):
    id_parts = df["ID"].str.split(".", expand=True)
    if id_parts.shape[1] >= 3:
        df["GENE"] = id_parts[0]
        df["MASK"] = id_parts[1]
        df["AAF_BIN"] = aaf_label
    else:
        print("⚠️  WARNING: ID format may not match expected GENE.MASK.AAF pattern")
        df["GENE"] = df["ID"]
        df["MASK"] = "unknown"
        df["AAF_BIN"] = aaf_label
    return df

# ── Handle duplicates ─────────────────────────────────
def filter_duplicates(df, strategy="best_mask"):
    if strategy == "keep_all":
        return df
    elif strategy == "highest_score":
        return df.loc[df.groupby("GENE")["score"].idxmax()].copy()
    elif strategy == "best_mask":
        mask_priority = {"M1": 1, "M2": 2, "M3": 3, "M4": 4}
        df["mask_priority"] = df["MASK"].map(mask_priority).fillna(99)
        best = df.loc[df.groupby("GENE")["mask_priority"].idxmin()].copy()
        return best.drop("mask_priority", axis=1)
    else:
        raise ValueError(f"Unknown duplicate strategy: {strategy}")

# ── Apply raw significance filtering ─────────────────────────────
def apply_raw_significance_filter(df, threshold):
    if threshold is None:
        return df

    # Convert LOG10P back to p-value for filtering
    if "LOG10P" in df.columns:
        df_filtered = df[df["LOG10P"] >= -np.log10(threshold)].copy()
        print(f"   Applied raw p < {threshold} filter: {len(df_filtered)} entries remain")
        return df_filtered
    else:
        print("⚠️  WARNING: Cannot apply significance filter without LOG10P column")
        return df

# ── Apply Bonferroni correction ─────────────────────────────
def apply_bonferroni_correction(df, alpha=0.05):
    if "LOG10P" not in df.columns:
        print("⚠️  WARNING: Cannot apply Bonferroni correction without LOG10P column")
        return df

    # Calculate raw p-values from LOG10P
    df["raw_pvalue"] = 10**(-df["LOG10P"])

    # Number of tests for Bonferroni correction
    n_tests = len(df)
    bonferroni_threshold = alpha / n_tests

    # Apply Bonferroni correction
    df["bonferroni_corrected_pvalue"] = df["raw_pvalue"] * n_tests
    df["bonferroni_corrected_pvalue"] = df["bonferroni_corrected_pvalue"].clip(upper=1.0)  # Cap at 1.0
    df["bonferroni_corrected_log10p"] = -np.log10(df["bonferroni_corrected_pvalue"])

    # Filter for Bonferroni significance
    df_significant = df[df["bonferroni_corrected_pvalue"] <= alpha].copy()

    print(f"   Bonferroni correction applied:")
    print(f"   • Number of tests: {n_tests}")
    print(f"   • Raw threshold: p < {bonferroni_threshold:.2e}")
    print(f"   • Family-wise alpha: {alpha}")
    print(f"   • Bonferroni significant: {len(df_significant)} entries")

    return df_significant, bonferroni_threshold

# ── Load regenie results ──────────────────────────────────────────
def main():
    print("\n" + "="*80)
    print("LOVO GENE SELECTION FROM REGENIE BURDEN TEST RESULTS")
    print("WITH BONFERRONI MULTIPLE TESTING CORRECTION")
    print("="*80)

    # ── validate parameters ──────────────────────────────────────────
    print("\n🔧 Validating parameters...")
    if TOP_N <= 0:
        print(f"❌ ERROR: TOP_N must be positive, got {TOP_N}")
        sys.exit(1)

    regenie_path = Path(REGENIE_FILE)
    if not regenie_path.exists():
        print(f"❌ ERROR: Input file not found: {REGENIE_FILE}")
        sys.exit(1)

    if SCORE_METHOD not in ["logp", "chisq", "beta"]:
        print(f"❌ ERROR: Invalid SCORE_METHOD: {SCORE_METHOD}")
        sys.exit(1)

    if DUPLICATE_STRATEGY not in ["best_mask", "keep_all", "highest_score"]:
        print(f"❌ ERROR: Invalid DUPLICATE_STRATEGY: {DUPLICATE_STRATEGY}")
        sys.exit(1)

    if APPLY_BONFERRONI and not (0 < BONFERRONI_ALPHA < 1):
        print(f"❌ ERROR: BONFERRONI_ALPHA must be between 0 and 1, got {BONFERRONI_ALPHA}")
        sys.exit(1)

    print("✅ All parameters validated")

    # ── load results ──────────────────────────────────────────
    print(f"\n📖 Loading REGENIE results from: {REGENIE_FILE}")
    try:
        df = pd.read_csv(REGENIE_FILE, sep=r"\s+", comment="#", header=0)
        print(f"✅ Loaded {len(df)} rows × {len(df.columns)} columns")
        print(f"   Available tests: {', '.join(df['TEST'].unique())}")
    except Exception as e:
        print(f"❌ ERROR loading file: {e}")
        sys.exit(1)

    # ── filter to test ──────────────────────────────────────────
    print(f"\n🔎 Filtering to test: {TEST_NAME}")
    df_test = df[df["TEST"] == TEST_NAME].copy()
    if df_test.empty:
        print(f"❌ ERROR: No results found for '{TEST_NAME}'")
        sys.exit(1)
    print(f"✅ Found {len(df_test)} entries for {TEST_NAME}")

    # ── extract gene and mask info ──────────────────────────────────────────
    print(f"\n🧬 Extracting gene and mask information...")
    df_test = extract_gene_mask_info(df_test, AAF_BIN_LABEL)
    print(f"✅ Found {df_test['GENE'].nunique()} unique genes")
    print("   Mask distribution:")
    for mask, count in df_test['MASK'].value_counts().items():
        print(f"     {mask}: {count} entries")

    # ── compute scores ──────────────────────────────────────────
    print(f"\n📊 Computing scores (method: {SCORE_METHOD})")
    df_scored = compute_score(df_test, SCORE_METHOD)
    if df_scored.empty:
        print(f"❌ ERROR: No valid scores with method '{SCORE_METHOD}'")
        sys.exit(1)
    print(f"✅ {len(df_scored)} entries with valid scores")
    print(f"   Score range: {df_scored['score'].min():.4f} → {df_scored['score'].max():.4f}")

    # ── apply raw significance filtering ──────────────────────────────────────────
    if RAW_SIGNIFICANCE_THRESHOLD is not None:
        print(f"\n🎯 Applying raw significance filtering (p < {RAW_SIGNIFICANCE_THRESHOLD})...")
        df_scored = apply_raw_significance_filter(df_scored, RAW_SIGNIFICANCE_THRESHOLD)
        if df_scored.empty:
            print(f"❌ ERROR: No significant results at raw p < {RAW_SIGNIFICANCE_THRESHOLD}")
            sys.exit(1)

    # ── apply bonferroni correction ──────────────────────────────────────────
    bonferroni_threshold = None
    if APPLY_BONFERRONI:
        print(f"\n🔬 Applying Bonferroni correction (α = {BONFERRONI_ALPHA})...")
        df_bonferroni, bonferroni_threshold = apply_bonferroni_correction(df_scored, BONFERRONI_ALPHA)

        if df_bonferroni.empty:
            print(f"⚠️  WARNING: No genes pass Bonferroni correction at α = {BONFERRONI_ALPHA}")
            print(f"   Proceeding with top {TOP_N} genes from original dataset for exploratory analysis")
            # Keep original data but add Bonferroni columns for reference
            df_scored["raw_pvalue"] = 10**(-df_scored["LOG10P"])
            df_scored["bonferroni_corrected_pvalue"] = df_scored["raw_pvalue"] * len(df_scored)
            df_scored["bonferroni_corrected_pvalue"] = df_scored["bonferroni_corrected_pvalue"].clip(upper=1.0)
            df_scored["bonferroni_corrected_log10p"] = -np.log10(df_scored["bonferroni_corrected_pvalue"])
            df_working = df_scored
        else:
            print(f"✅ {len(df_bonferroni)} genes pass Bonferroni correction")
            df_working = df_bonferroni
    else:
        print("\n🔬 Skipping Bonferroni correction (APPLY_BONFERRONI = False)")
        df_working = df_scored

    # ── handle duplicates ──────────────────────────────────────────
    print(f"\n🔄 Handling duplicate genes (strategy: {DUPLICATE_STRATEGY})...")
    initial_count = len(df_working)
    unique_genes_initial = df_working['GENE'].nunique()

    df_working = filter_duplicates(df_working, strategy=DUPLICATE_STRATEGY)

    final_count = len(df_working)
    unique_genes_final = df_working['GENE'].nunique()

    print(f"✅ Deduplication complete:")
    print(f"   Before: {initial_count} entries, {unique_genes_initial} unique genes")
    print(f"   After:  {final_count} entries, {unique_genes_final} unique genes")

    # ── select top entries ──────────────────────────────────────────
    print(f"\n🏆 Selecting top {TOP_N} entries")
    df_top = df_working.sort_values("score", ascending=False).head(TOP_N)

    print("\nTop gene–mask combinations:")
    print("-" * 100)
    score_label = {"logp": "LOG10P", "chisq": "CHISQ", "beta": "|BETA|"}[SCORE_METHOD]

    for i, (_, row) in enumerate(df_top.iterrows(), 1):
        bonf_info = ""
        if APPLY_BONFERRONI and "bonferroni_corrected_pvalue" in row:
            bonf_p = row["bonferroni_corrected_pvalue"]
            significance = "***" if bonf_p <= BONFERRONI_ALPHA else ""
            bonf_info = f" | Bonf.p={bonf_p:.2e}{significance}"

        print(f"{i:2d}. {row['GENE']:<12} ({row['MASK']}) - {score_label}={row['score']:.4f}{bonf_info}")

    # ── write outputs ──────────────────────────────────────────
    print(f"\n💾 Writing LOVO gene list → {OUTPUT_FILE}")
    with open(OUTPUT_FILE, "w") as fout:
        # Write header
        fout.write("GENE,MASK,AAF_BIN\n")
        for _, row in df_top.iterrows():
            fout.write(f"{row['GENE']},{row['MASK']},{row['AAF_BIN']}\n")
    print(f"✅ Saved {len(df_top)} entries to {OUTPUT_FILE}")

    # Write detailed results file
    detailed_output = OUTPUT_FILE.replace(".txt", "_detailed.csv")
    output_cols = ["GENE", "MASK", "AAF_BIN", "BETA", "LOG10P", "CHISQ", "score"]
    if APPLY_BONFERRONI and "bonferroni_corrected_pvalue" in df_top.columns:
        output_cols.extend(["raw_pvalue", "bonferroni_corrected_pvalue", "bonferroni_corrected_log10p"])

    available_cols = [col for col in output_cols if col in df_top.columns]
    df_top[available_cols].to_csv(detailed_output, index=False)
    print(f"✅ Detailed results saved to {detailed_output}")

    # ── summary ──────────────────────────────────────────
    print("\n" + "="*60)
    print("SUMMARY")
    print("="*60)
    print(f"📂 Input file: {REGENIE_FILE}")
    print(f"🧪 Test analysed: {TEST_NAME}")
    print(f"📊 Scoring method: {SCORE_METHOD}")
    print(f"🔄 Duplicate strategy: {DUPLICATE_STRATEGY}")
    print(f"🎯 Raw significance filter: {RAW_SIGNIFICANCE_THRESHOLD if RAW_SIGNIFICANCE_THRESHOLD else 'None'}")
    print(f"🔬 Bonferroni correction: {'Yes' if APPLY_BONFERRONI else 'No'}")
    if APPLY_BONFERRONI:
        print(f"   • Family-wise α: {BONFERRONI_ALPHA}")
        if bonferroni_threshold:
            print(f"   • Raw p-value threshold: {bonferroni_threshold:.2e}")
    print(f"📈 Total entries analysed: {len(df_working)}")
    print(f"🏆 Top entries selected: {TOP_N}")
    print(f"💾 Output file: {OUTPUT_FILE}")
    print(f"📋 Detailed results: {detailed_output}")

    # ── detailed results ──────────────────────────────────────────
    display_cols = ["GENE", "MASK", "AAF_BIN", "BETA", "LOG10P", "CHISQ"]
    if APPLY_BONFERRONI and "bonferroni_corrected_pvalue" in df_top.columns:
        display_cols.append("bonferroni_corrected_pvalue")

    available_display_cols = [col for col in display_cols if col in df_top.columns]
    print("\n📋 Detailed results:")
    print(df_top[available_display_cols].to_string(index=False, float_format="%.4f"))

    print("\nLOVO gene list format (GENE,MASK,AAF_BIN):")
    for _, row in df_top.iterrows():
        print(f"{row['GENE']},{row['MASK']},{row['AAF_BIN']}")

    # ── bonferroni warning ──────────────────────────────────────────
    if APPLY_BONFERRONI:
        significant_genes = len(df_working[df_working["bonferroni_corrected_pvalue"] <= BONFERRONI_ALPHA]) if "bonferroni_corrected_pvalue" in df_working.columns else 0

        print(f"\n🔬 BONFERRONI CORRECTION SUMMARY:")
        print(f"   • Total tests performed: {len(df_scored)}")
        print(f"   • Genes passing correction: {significant_genes}")

        if significant_genes == 0:
            print("   ⚠️  No genes pass multiple testing correction")
        elif significant_genes < TOP_N:
            print(f"   ⚠️  Only {significant_genes} genes pass correction (< TOP_N)")

if __name__ == "__main__":
    main()